Análisis de errores en conjunto de validación
=============================================

## Introducción

El análisis de errores es el proceso para identificar, observar y diagnosticar predicciones erróneas de un modelo de aprendizaje automático, ayudandonos a comprender las áreas con fortalezas o debilidades de un modelo. Cuando decimos que "la precisión del modelo es del 90%", puede que no sea uniforme en todos los subgrupos de datos o puede haber algunas condiciones en los datos de entrada en las que el modelo falla más. Por lo tanto, es importante someter las métricas a una revisión más profunda para poder mejorarlo.

## Utilizando el análisis de errores en el problema censo de la UCI

> Nota: Este ejemplo fué adaptado de https://github.com/microsoft/responsible-ai-widgets/blob/main/notebooks/erroranalysis-interpretability-dashboard-census.ipynb

### Instalación

Utilizaremos las librerías `interpret-community`, `raiwidgets`. La instalación de estos paquetes requiere de el compilador g++. Si no lo tiene instalado, puede hacerlo desde:

```
apt install g++
```

Necesitaremos instalar las librerias `interpret-community`, `raiwidgets` y `lightgbm`

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/E72102/master/docs/develop/modeling/selection/code/error_analysis.txt \
    --quiet --no-clobber
!pip install -r error_analysis.txt --quiet

### Sobre el conjunto de datos del censo UCI

El conjunto de datos del censo de la UCI es un conjunto de datos en el que cada registro representa a una persona. Cada registro contiene 14 columnas que describen a una una sola persona, de la base de datos del censo de Estados Unidos de 1994. Esto incluye información como la edad, el estado civil y el nivel educativo. La tarea es determinar si una persona tiene un ingreso alto (definido como ganar más de $50 mil al año). Esta tarea, dado el tipo de datos que utiliza, se usa a menudo en el estudio de equidad, en parte debido a los atributos comprensibles del conjunto de datos, incluidos algunos que contienen tipos sensibles como la edad y el género, y en parte también porque comprende una tarea claramente del mundo real.

Descargamos el conjunto de datos

In [1]:
!wget https://santiagxf.blob.core.windows.net/public/datasets/uci_census.zip \
    --quiet --no-clobber
!mkdir -p datasets
!unzip -qq uci_census.zip -d datasets

Lo importamos

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('datasets/uci_census/data/adult-train.csv')
test = pd.read_csv('datasets/uci_census/data/adult-test.csv')

Generaremos 3 conjuntos de datos: entrenamiento, validación y testing.

In [2]:
from sklearn.model_selection import train_test_split

validation = test
test, _ = train_test_split(test, test_size=0.9, random_state=1234)

### Entrenando un modelo para explorar

Preparando nuestros conjuntos de datos

In [3]:
X_train = train.drop(['income'], axis=1)
y_train = train['income'].to_numpy()
X_test = train.drop(['income'], axis=1)
y_test = train['income'].to_numpy()
X_val = validation.drop(['income'], axis=1)
y_val = validation['income'].to_numpy()

In [4]:
classes = train['income'].unique().tolist()
features = X_train.columns.values.tolist()
categorical_features = X_train.dtypes[X_train.dtypes == 'object'].index.tolist()

Realizaremos un pequeño preprocesamiento antes de entrenar el modelo:

- Imputaremos los valores faltantes de las caracteristicas numéricas con la media
- Imputaremos los valores faltantes de las caracteristicas categóricas con el valor `?`
- Escalaremos los valores numericos utilizando un `StandardScaler`
- Codificaremos las variables categóricas utilizando `OneHotEncoder`

In [5]:
from typing import Tuple, List

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


def prepare(X: pd.DataFrame) -> Tuple[np.ndarray, sklearn.compose.ColumnTransformer]:
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    
    transformations = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = transformations.fit_transform(X)
    
    return X, transformations


X_train_transformed, transformations = prepare(X_train)
X_test_transformed = transformations.transform(X_test)

Entrenamos un modelo basado en `lightgbm`

In [6]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(n_estimators=1)
model = clf.fit(X_train_transformed, y_train)

Configuramos un objeto para general las explicaciones del modelo basado en el conjunto de datos en el que se entreno:

> **IMPORTANTE:** Note que esta ténica require la creación de un modelo que sea interpretable. Es decir, en lugar de realizar el análisis en el modelo original (el cual podría tener una complejidad arbitraria), el análsis se hace sobre un modelo que pueda ser facilmente interpretable. Para generar este segundo modelo, se utiliza la técnica de Global Model Surrogate la cual consiste en entrenar un modelo **alumno** que trata de **imitar** al modelo **profesor**. Esta técnica claramente no es exacta y no tenemos ninguna garantía de que los errores que comete el modelo **alumno** son los mismos que los que comete el alumno **profesor**.

In [57]:
from interpret_community.common.constants import ShapValuesOutput, ModelTask
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel


explainer = MimicExplainer(model=model,
                           initialization_examples=X_train,
                           explainable_model=LGBMExplainableModel,
                           augment_data=True, 
                           max_num_of_augmentations=10,
                           features=features, 
                           classes=classes, 
                           model_task=ModelTask.Classification,
                           transformations=transformations)

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


Generamos las explicaciones del modelo en nuestro conjunto de validación

In [58]:
global_explanation = explainer.explain_global(X_test)

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


### Análisis de errores

Para generar el análisis de errores, deberemos constuir un pipeline donde tengamos el preprocesamiento y el modelo propiamente dicho en un mismo objeto:

In [59]:
model_pipeline = Pipeline(steps=[('preprocessing', transformations),
                                 ('model', model)])

Abrimos la herramienta de análisis de errores

In [60]:
from raiwidgets import ErrorAnalysisDashboard

In [ ]:
ErrorAnalysisDashboard(global_explanation, model_pipeline, 
                       dataset=X_val,
                       true_y_dataset=y_val,
                       true_y=y_test, 
                       categorical_features=categorical_features)

Using categorical_feature in Dataset.
Removing duplicate bin edges for quantile binning of feature education-num. There are too many duplicate values for the specified number of bins.


Una vez ejecutado el comando anterior, debería poder ver la herramienta de exloración de errores:

<img src="../../../_images/ea_treemap.png" />

#### Interpretación

Podemos utilizar este gráfico para explorar la forma en que el modelo comente los errores. Para encontrar patrones, podemos ocmenzar buscando aquellos nodos en el arbol que tienen un color rojo más fuerte, lo que indica que esa combinación de atributos tiene un error alto al clasificarlos. El nivel de llenado del nodo indica que tan representativa es esa combinación de atributos en el conjunto de datos completo

Esto quiere decir que si nos focalizamos en aquellos nodos con color más oscuro y nivel más alto, estamos atacando aquellas áreas donde tenemos más chances de mejorar la performance del modelo. Por ejemplo, en la imagen anterior vemos que cuando la relación es `Husband` o `Wife` y la cantidad de años de educación es mayor a 11.5 pero el capital es menor a $ 5035.50, estas instancias tienen una taza de error del 65% y representan el 35% de todos los errores que comete el modelo.

Deberiamos investigar porque nuestro modelo no puede mapear a este tipo de instancias correctamente. Quizás haya un probema en la calidad de datos, una mala recolección, o quizás las características no fueron preprocesadas correctamente.

#### Instancias con dificultades

Una característica interesante de esta libreria es la capacidad de generar mapas de calor con aquellas combinaciones de atributos donde nuestro modelo tiene problemas. Esto nos permite ver rapidamente donde el modelo tiene inconvenientes en predecir correctamente y desde allí, analizár si el modelo es aceptable cometiendo estos errores o no:

<img src="../../../_images/ea_heatmap.png" width="600" />

En el ejemplo más arriba estamos comparando los predictores `relationship` y `education-num`. Como vemos, el modelo tiene grandes problemas con aquellas personas de más de 14 años de educación y que son mujeres casadas. Solo 1 persona fué clasificada correctamente representando una taza de error del 94%.